# Using the RubinTV Mount Plotting to quantify mount errors

Craig Lage - 06-Oct-22

In [ ]:
import nest_asyncio
nest_asyncio.apply()
import sys, time, os, asyncio
from datetime import datetime
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
from lsst.rubintv.production import mountTorques

from lsst_efd_client import EfdClient
from lsst.daf.butler import Butler
from lsst.ts.observing.utilities.decorated_logger import DecoratedLogger

In [ ]:
client = EfdClient('summit_efd')
butler = Butler('/repo/LATISS', collections=["LATISS/raw/all", "LATISS/calib"])
logger = DecoratedLogger.get_decorated_logger()

In [ ]:
figure = plt.figure(figsize=(16,22))
saveFilename = None
dayObss = [20230404]
errs = []
for dayObs in dayObss:
    exposureList = []
    for record in butler.registry.queryDimensionRecords("exposure", where="exposure.day_obs=%d"%dayObs):
        #print(record.id, record.observation_type)
        if record.observation_type in ['dark']:
            try:
                exposureList.append([record.id, 90.0-record.zenith_angle, record.azimuth])
            except:
                continue
    print("ExpId              Elevation       Azimuth         Mount Error")
    for [expId,alt,az] in exposureList:
        dataId = {'detector':0, 'exposure':expId}
        saveFileName = f"/home/craiglagegit/DATA/stiction_04apr23/Mount_Graph_{expId}.pdf"
        err = mountTorques.calculateMountErrors(dataId, butler, client, figure, saveFileName, logger)
        print(f"{expId}       {alt:.2f}          {az:.2f}              {err['image_az_rms']:.3f}")
